In [1]:
# from neuralprophet import NeuralProphet
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from utils import generate_lagged_features, scale_data
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from utils import mape, gmrae, smape_adjusted, nrmse
from sklearn.linear_model import ElasticNet


from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

/Users/pantelisdogoulis/miniforge3/envs/energy-forecasting/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
lagged_df = pd.read_csv("lagged_df_kw.csv", parse_dates=["datetime"])
lagged_df.set_index('datetime', inplace=True)
lagged_df["AP"] = lagged_df["AP"].astype("category")
lagged_df["AP"] = lagged_df["AP"].cat.codes

In [4]:
test = lagged_df[lagged_df.index.year==2021]
train = lagged_df[lagged_df.index.year.isin([2015,2016,2017,2018,2019,2020])]

In [5]:
N_LAGS = 5
N_FORECASTS = 4 # (FORECASTS-1)

create DF:

In [6]:
X_features = [f'lag{i}' for i in range(N_FORECASTS+1,N_FORECASTS+1+N_LAGS)]
y_features = [f'lag{i}' for i in range(0, N_FORECASTS+1)]

In [24]:
metrics_df = pd.DataFrame()

y_pred_df = pd.DataFrame()

for cluster in tqdm(lagged_df.clusters.unique()):

    X_features = [f'lag{i}' for i in range(N_FORECASTS+1,N_FORECASTS+1+N_LAGS)]

    # TRAIN:
    train_cluster = train[train['clusters']==cluster]
    train_cluster['mean'] = train_cluster[X_features].mean(axis=1)
    train_cluster['std'] = train_cluster[X_features].std(axis=1)
    train_cluster['skew'] = train_cluster[X_features].skew(axis=1)
    train_cluster['kurtosis'] = train_cluster[X_features].kurtosis(axis=1)
    train_cluster['max'] = train_cluster[X_features].max(axis=1)
    train_cluster['min'] = train_cluster[X_features].min(axis=1)
    train_cluster['mean'] = train_cluster[X_features].mean(axis=1)
    # train_cluster['mean_last_2'] = train_cluster[['lag6', 'lag5']].mean(axis=1)
    # train_cluster['mean_past_2'] = train_cluster[['lag9', 'lag8']].mean(axis=1)
    train_cluster['std'] = train_cluster[X_features].std(axis=1)
    train_cluster['var'] = train_cluster[X_features].var(axis=1)


    # PREDICT
    test_cluster = test[test['clusters']==cluster]
    
    test_cluster['max'] = test_cluster[X_features].max(axis=1)
    test_cluster['min'] = test_cluster[X_features].min(axis=1)
    test_cluster['mean'] = test_cluster[X_features].mean(axis=1)
    # test_cluster['mean_last_2'] = test_cluster[['lag6', 'lag5']].mean(axis=1)
    # test_cluster['mean_past_2'] = test_cluster[['lag9', 'lag8']].mean(axis=1)
    test_cluster['std'] = test_cluster[X_features].std(axis=1)
    test_cluster['skew'] = test_cluster[X_features].skew(axis=1)
    test_cluster['kurtosis'] = test_cluster[X_features].kurtosis(axis=1)
    test_cluster['var'] = test_cluster[X_features].var(axis=1)

    ad_features = ['mean', 'min','std', 'skew', 'kurtosis','sin_month', 'cos_month', 'sin_dayofyear', 'cos_dayofyear', 'sin_week',
       'cos_week', 'max'] 
    X_features.extend(ad_features)

    X_train = train_cluster[X_features]
    y_train = train_cluster[y_features]

    # reg = MultiOutputRegressor(LGBMRegressor(n_estimators=400, max_depth=8, min_child_samples=20, reg_lambda=0.1, min_split_gain=20, num_leaves=31))
    reg = ElasticNet()
    reg.fit(X_train,  y_train)

    # PREDICT

    for customer_id in tqdm(test_cluster.AP.unique()):
            
        customer_cluster = test_cluster[test_cluster['AP']==customer_id]
        X_test = customer_cluster[X_features]
        y_test = customer_cluster[y_features]

        y_pred = reg.predict(X_test)

        y_pred = pd.DataFrame(y_pred)
        y_pred.columns = y_features

        # NAIVE
        y_naive = pd.DataFrame(X_test[['lag5']*5])
        y_naive.index = X_test.index
        y_naive.columns = y_test.columns


        # metrics
    
        mse = mean_squared_error(y_test.values, y_pred)
        mae = mean_absolute_error(y_test.values, y_pred)
        mape_sk = mean_absolute_percentage_error(y_test.values, y_pred)
        r_sq = r2_score(y_test.values, y_pred)
        nrmae = mean_absolute_error(y_test, y_pred) / mean_absolute_error(y_test, y_naive)
        mape_value = np.mean(mape(y_test.values, y_pred))
        smape_value = np.mean(smape_adjusted(y_test.values, y_pred))
        # mape_value = mape(y_test.values, y_pred.values)
        # gmrae_value = gmrae(y_test.values, y_pred.values, naive.values)
        # smape_value = smape_adjusted(y_test.values, y_pred.values)

        metrics = np.concatenate((np.reshape(customer_id, (1,1)),np.reshape(mse, (1,1)), \
        np.reshape(mae, (1,1)), np.reshape(mape_sk, (1,1)), np.reshape(r_sq, (1,1)),\
        np.reshape(mape_value, (1,1)), np.reshape(smape_value, (1,1)), np.reshape(nrmae, (1,1)) ), axis=1)
        metrics = pd.DataFrame(metrics)
        metrics.columns = ['AP', 'mse', 'mae', 'mape_sk', 'r_sq', 'mape', 'smape', 'nrmae']

        metrics_df = pd.concat((metrics_df, metrics), axis=0)
        
        customer_id_array = np.full((39,1), customer_id)
        y_pred = np.concatenate((y_pred, customer_id_array),1)
        y_pred = pd.DataFrame(y_pred)
        y_pred.columns = ['lag4', 'lag3', 'lag2', 'lag1', 'lag0', 'AP']
        y_pred.index = y_test.index

        y_pred_df = pd.concat((y_pred_df, y_pred), axis=0)


100%|██████████| 9/9 [01:18<00:00,  8.74s/it]
